<a href="https://colab.research.google.com/github/ArkS0001/FlipkartPrep/blob/main/EncryptionALGOS_Comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import time
from cryptography.fernet import Fernet
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import hashes
from argon2 import PasswordHasher
import hmac
import hashlib
import os

# Benchmarking Function
def benchmark(func, *args):
    start_time = time.time()
    result = func(*args)
    end_time = time.time()
    return result, end_time - start_time

# Data for benchmarking
data = b"Benchmarking cryptographic algorithms"
password = "my_secure_password"
key = os.urandom(32)

# 1. Fernet
def fernet_test(data):
    fernet_key = Fernet.generate_key()
    fernet = Fernet(fernet_key)
    encrypted = fernet.encrypt(data)
    decrypted = fernet.decrypt(encrypted)
    return decrypted

fernet_result, fernet_time = benchmark(fernet_test, data)

# 2. AES-GCM
def aes_gcm_test(data):
    aes_key = os.urandom(32)
    aes_nonce = os.urandom(12)
    cipher = Cipher(algorithms.AES(aes_key), modes.GCM(aes_nonce))
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(data) + encryptor.finalize()
    tag = encryptor.tag
    cipher = Cipher(algorithms.AES(aes_key), modes.GCM(aes_nonce, tag))
    decryptor = cipher.decryptor()
    decrypted = decryptor.update(ciphertext) + decryptor.finalize()
    return decrypted

aes_gcm_result, aes_gcm_time = benchmark(aes_gcm_test, data)

# 3. RSA
def rsa_test(data):
    rsa_private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048)
    rsa_public_key = rsa_private_key.public_key()
    ciphertext = rsa_public_key.encrypt(
        data,
        padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA256()), algorithm=hashes.SHA256(), label=None),
    )
    decrypted = rsa_private_key.decrypt(
        ciphertext,
        padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA256()), algorithm=hashes.SHA256(), label=None),
    )
    return decrypted

rsa_result, rsa_time = benchmark(rsa_test, data)

# 4. HMAC
def hmac_test(data):
    hmac_key = os.urandom(32)
    h = hmac.new(hmac_key, data, hashlib.sha256)
    digest = h.hexdigest()
    return digest

hmac_result, hmac_time = benchmark(hmac_test, data)

# 5. ChaCha20
def chacha20_test(data):
    chacha_key = os.urandom(32)
    chacha_nonce = os.urandom(16)
    cipher = Cipher(algorithms.ChaCha20(chacha_key, chacha_nonce), mode=None)
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(data)
    decryptor = cipher.decryptor()
    decrypted = decryptor.update(ciphertext)
    return decrypted

chacha_result, chacha_time = benchmark(chacha20_test, data)

# 6. Argon2
def argon2_test(password):
    ph = PasswordHasher()
    hashed = ph.hash(password)
    verified = ph.verify(hashed, password)
    return verified

argon2_result, argon2_time = benchmark(argon2_test, password)

# Display Results
print("\n--- Performance Comparison ---")
print(f"Fernet: {fernet_time:.6f} seconds")
print(f"AES-GCM: {aes_gcm_time:.6f} seconds")
print(f"RSA: {rsa_time:.6f} seconds")
print(f"HMAC-SHA256: {hmac_time:.6f} seconds")
print(f"ChaCha20: {chacha_time:.6f} seconds")
print(f"Argon2 (Password Hashing): {argon2_time:.6f} seconds")


--- Performance Comparison ---
Fernet: 0.004974 seconds
AES-GCM: 0.000170 seconds
RSA: 0.041849 seconds
HMAC-SHA256: 0.000043 seconds
ChaCha20: 0.000129 seconds
Argon2 (Password Hashing): 0.404415 seconds


In [2]:
import time
from cryptography.fernet import Fernet
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import hashes
from argon2 import PasswordHasher
import hmac
import hashlib
import os

# Benchmarking Function
def benchmark(func, *args):
    start_time = time.time()
    result = func(*args)
    end_time = time.time()
    return result, end_time - start_time

# Data for benchmarking
data = b"Benchmarking cryptographic algorithms"
password = "my_secure_password"
key = os.urandom(32)

# Results dictionary
results = {}

# 1. Fernet
def fernet_test(data):
    fernet_key = Fernet.generate_key()
    fernet = Fernet(fernet_key)
    enc_start = time.time()
    encrypted = fernet.encrypt(data)
    enc_end = time.time()
    dec_start = time.time()
    decrypted = fernet.decrypt(encrypted)
    dec_end = time.time()
    return enc_end - enc_start, dec_end - dec_start, len(encrypted), len(fernet_key)

fernet_enc_time, fernet_dec_time, fernet_output_size, fernet_key_size = fernet_test(data)
results["Fernet"] = (fernet_enc_time, fernet_dec_time, fernet_output_size, fernet_key_size, "128-bit AES + HMAC", "Symmetric encryption")

# 2. AES-GCM
def aes_gcm_test(data):
    aes_key = os.urandom(32)
    aes_nonce = os.urandom(12)
    cipher = Cipher(algorithms.AES(aes_key), modes.GCM(aes_nonce))
    enc_start = time.time()
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(data) + encryptor.finalize()
    enc_end = time.time()
    tag = encryptor.tag
    cipher = Cipher(algorithms.AES(aes_key), modes.GCM(aes_nonce, tag))
    dec_start = time.time()
    decryptor = cipher.decryptor()
    decrypted = decryptor.update(ciphertext) + decryptor.finalize()
    dec_end = time.time()
    return enc_end - enc_start, dec_end - dec_start, len(ciphertext) + len(tag), len(aes_key)

aes_gcm_enc_time, aes_gcm_dec_time, aes_gcm_output_size, aes_gcm_key_size = aes_gcm_test(data)
results["AES-GCM"] = (aes_gcm_enc_time, aes_gcm_dec_time, aes_gcm_output_size, aes_gcm_key_size, "128/192/256-bit AES", "Symmetric authenticated encryption")

# 3. RSA
def rsa_test(data):
    rsa_private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048)
    rsa_public_key = rsa_private_key.public_key()
    enc_start = time.time()
    ciphertext = rsa_public_key.encrypt(
        data,
        padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA256()), algorithm=hashes.SHA256(), label=None),
    )
    enc_end = time.time()
    dec_start = time.time()
    decrypted = rsa_private_key.decrypt(
        ciphertext,
        padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA256()), algorithm=hashes.SHA256(), label=None),
    )
    dec_end = time.time()
    return enc_end - enc_start, dec_end - dec_start, len(ciphertext), 2048 // 8

rsa_enc_time, rsa_dec_time, rsa_output_size, rsa_key_size = rsa_test(data)
results["RSA"] = (rsa_enc_time, rsa_dec_time, rsa_output_size, rsa_key_size, "2048-bit keys", "Asymmetric encryption and signing")

# 4. ChaCha20
def chacha20_test(data):
    chacha_key = os.urandom(32)
    chacha_nonce = os.urandom(16)
    cipher = Cipher(algorithms.ChaCha20(chacha_key, chacha_nonce), mode=None)
    enc_start = time.time()
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(data)
    enc_end = time.time()
    dec_start = time.time()
    decryptor = cipher.decryptor()
    decrypted = decryptor.update(ciphertext)
    dec_end = time.time()
    return enc_end - enc_start, dec_end - dec_start, len(ciphertext), len(chacha_key)

chacha_enc_time, chacha_dec_time, chacha_output_size, chacha_key_size = chacha20_test(data)
results["ChaCha20"] = (chacha_enc_time, chacha_dec_time, chacha_output_size, chacha_key_size, "256-bit keys", "Stream cipher")

# 5. Argon2
def argon2_test(password):
    ph = PasswordHasher()
    start = time.time()
    hashed = ph.hash(password)
    end = time.time()
    return end - start, len(hashed)

argon2_time, argon2_output_size = argon2_test(password)
results["Argon2"] = (argon2_time, None, argon2_output_size, None, "Password hashing", "Resistant to brute force")

# Display Results
print("\n--- Detailed Performance Comparison ---")
print(f"{'Algorithm':<10}{'Enc Time':<12}{'Dec Time':<12}{'Output Size':<14}{'Key Size':<10}{'Description':<30}{'Use Case'}")
for algo, (enc_time, dec_time, output_size, key_size, description, use_case) in results.items():
    print(f"{algo:<10}{enc_time:.6f}s   {dec_time if dec_time else 'N/A':<12}{output_size:<14}{key_size if key_size else 'N/A':<10}{description:<30}{use_case}")


--- Detailed Performance Comparison ---
Algorithm Enc Time    Dec Time    Output Size   Key Size  Description                   Use Case
Fernet    0.000169s   6.961822509765625e-05140           44        128-bit AES + HMAC            Symmetric encryption
AES-GCM   0.000037s   1.8835067749023438e-0553            32        128/192/256-bit AES           Symmetric authenticated encryption
RSA       0.000144s   0.0013883113861083984256           256       2048-bit keys                 Asymmetric encryption and signing
ChaCha20  0.000051s   1.0728836059570312e-0537            32        256-bit keys                  Stream cipher
Argon2    0.170061s   N/A         97            N/A       Password hashing              Resistant to brute force
